In [31]:
import pandas as pd

In [32]:
# train_PCA_YZ_df = pd.read_pickle("D:/Oxford/Second Year/AutoGeTS/Data/Train_PCA_YZ_withPred_0.pkl")

# topic_dict = {"T1": "IT support and assistance.","T2": "Account activation and access issues.","T3": "Password and device security.",
#                     "T4": "Printer issues and troubleshooting.","T5": "HP Dock connectivity issues.","T6": "Employee documentation and errors.",
#                     "T7": "\"Access and login issues\"","T8": "Opening and managing files/devices.","T9": "Mobile email and VPN setup.",
#                     "T10": "IT support and communication.","T11": "Error handling in RPG programming.","T12": "Email security and attachments.",
#                     "T13": "Humanitarian aid for Ukraine.","T14": "Internet connectivity issues in offices.","T15": "Improving integration with Infojobs."}

In [33]:
"""TREC-6"""
# train_PCA_YZ_df = pd.read_pickle("D:/Oxford/Second Year/AutoGeTS_Revisions/Additional_Datasets/TREC-6/Data/Train_PCA_YZ_withPred_0.pkl")

# topic_dict = {'ABBR': 'ABBR', 'DESC': 'DESC', 'ENTY': 'ENTY', 'HUM': 'HUM', 'LOC': 'LOC', 'NUM': 'NUM'}

'TREC-6'

In [34]:
"""Amazon Review"""
train_PCA_YZ_df = pd.read_pickle("D:/Oxford/Second Year/AutoGeTS_Revisions/Additional_Datasets/Amazon_Review/Data/Train_PCA_YZ_withPred_0.pkl")

topic_dict = {'R1': '1', 'R2': '2', 'R3': '3', 'R4': '4', 'R5': '5'}

In [35]:
import pandas as pd
import os
from nlpaug.augmenter.word import SynonymAug
from transformers import MarianMTModel, MarianTokenizer
from deep_translator import GoogleTranslator
import numpy as np
from tqdm import tqdm
import pickle

import random
import torch
import time

In [36]:
def set_global_seeds(seed=42):
    """Set seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [37]:
# """For Inetum"""
# # Add this at the beginning of the script
# class Config:
#     """Configuration class for save directories"""
#     def __init__(self, base_dir="synthetic_data"):
#         self.base_dir = base_dir
#         self.eda_dir = os.path.join(base_dir, "EDA")
#         self.bt_dir = os.path.join(base_dir, "Back-translation")

# def setup_folders(config):
#     """Create necessary folders for storing results"""
#     os.makedirs(config.base_dir, exist_ok=True)
#     os.makedirs(config.eda_dir, exist_ok=True)
#     os.makedirs(config.bt_dir, exist_ok=True)

# class TopicDataFrame:
#     """Class to handle topic dataframes with incremental saves in both CSV and PKL formats"""
#     def __init__(self, topic_num, save_dir):
#         self.topic_num = topic_num
#         self.save_dir = save_dir
#         self.csv_filename = os.path.join(save_dir, f"{topic_num}_synthetic.csv")
#         self.pkl_filename = os.path.join(save_dir, f"{topic_num}_synthetic.pkl")
#         self.columns = ['index_meta', 'topic_name', 'sample', 'area_TEIS', 'text']
        
#         # Load existing data if any, otherwise create new DataFrame
#         if os.path.exists(self.pkl_filename):
#             with open(self.pkl_filename, 'rb') as f:
#                 self.df = pickle.load(f)
#         elif os.path.exists(self.csv_filename):
#             self.df = pd.read_csv(self.csv_filename)
#         else:
#             self.df = pd.DataFrame(columns=self.columns)
    
#     def append_and_save(self, new_row):
#         """Append a new row and save immediately in both formats"""
#         # Convert new_row dictionary to DataFrame
#         new_row_df = pd.DataFrame([new_row])
        
#         # Concatenate with existing DataFrame
#         self.df = pd.concat([self.df, new_row_df], ignore_index=True)
        
#         # Save as CSV
#         self.df.to_csv(self.csv_filename, index=False)
        
#         # Save as PKL
#         with open(self.pkl_filename, 'wb') as f:
#             pickle.dump(self.df, f)

# def initialize_topic_dfs(topic_dict, save_dir):
#     """Initialize TopicDataFrame objects for each topic"""
#     topic_dfs = {}
#     for topic_num in topic_dict.keys():
#         topic_dfs[topic_num] = TopicDataFrame(topic_num, save_dir)
#     return topic_dfs

# def setup_language_models():
#     """No need to setup models with deep-translator"""
#     return None

# # def eda_augment(text, num_aug=5):
# #     """Perform EDA augmentation for Spanish/Portuguese text"""
# #     # Initialize augmenter with Spanish WordNet
# #     aug = SynonymAug(aug_src='wordnet', lang='spa')  # Use 'por' for Portuguese
    
# #     try:
# #         augmented_texts = aug.augment(text, n=num_aug)
# #         return augmented_texts
# #     except:
# #         # Fallback: if augmentation fails, return slight modifications of original text
# #         return [text + " " + suffix for suffix in [
# #             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
# #         ]]

# # def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5):
# #     """Perform back-translation augmentation"""
# #     augmented_texts = []
    
# #     try:
# #         # Translate to English
# #         en_text = model_to_en.generate(
# #             **tokenizer_to_en(text, return_tensors="pt", padding=True),
# #             num_beams=5,
# #             num_return_sequences=num_aug,
# #             max_length=256
# #         )
        
# #         # Translate back to Spanish/Portuguese
# #         for en in en_text:
# #             es_text = model_to_es.generate(
# #                 **tokenizer_to_es(tokenizer_to_en.decode(en, skip_special_tokens=True), 
# #                                 return_tensors="pt", padding=True),
# #                 num_beams=1,
# #                 max_length=256
# #             )
# #             augmented_texts.append(tokenizer_to_es.decode(es_text[0], skip_special_tokens=True))
            
# #         return augmented_texts
# #     except:
# #         # Fallback: if translation fails, return slight modifications
# #         return [text + " " + suffix for suffix in [
# #             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
# #         ]]

# def eda_augment(text, num_aug=5, alpha=0.1, seed=42):
#     """
#     Perform Easy Data Augmentation for Spanish/Portuguese text with robust error handling
#     """
#     # Set seed
#     random.seed(seed)
#     np.random.seed(seed)

#     # Add input validation
#     if not isinstance(text, str) or not text.strip():
#         return [text] * num_aug
    
#     def get_synonyms(word, lang='spa'):  # Use 'por' for Portuguese
#         try:
#             aug = SynonymAug(aug_src='wordnet', lang=lang)
#             synonyms = aug.augment(word)[0]
#             return synonyms if isinstance(synonyms, list) else [synonyms]
#         except:
#             return [word]
    
#     def synonym_replacement(words, n):
#         if not words:
#             return words
#         new_words = words.copy()
#         random_word_list = list(set([word for word in words if word.isalnum()]))
#         if not random_word_list:  # If no alphanumeric words found
#             return words
#         n = min(n, len(random_word_list))
#         random_idx = random.sample(range(len(random_word_list)), n)
        
#         for idx in random_idx:
#             word = random_word_list[idx]
#             synonyms = get_synonyms(word)
#             if synonyms:
#                 random_syn = random.choice(synonyms)
#                 new_words = [random_syn if w == word else w for w in new_words]
#         return new_words
    
#     def random_insertion(words, n):
#         if not words:
#             return words
#         new_words = words.copy()
#         # Get list of alphanumeric words
#         alnum_words = [w for w in words if w.isalnum()]
#         if not alnum_words:  # If no alphanumeric words found
#             return words
            
#         for _ in range(n):
#             try:
#                 random_word = random.choice(alnum_words)
#                 synonyms = get_synonyms(random_word)
#                 if synonyms:
#                     random_syn = random.choice(synonyms)
#                     random_idx = random.randint(0, len(new_words))
#                     new_words.insert(random_idx, random_syn)
#             except Exception as e:
#                 print(f"Warning: random_insertion failed with error {str(e)}")
#                 continue
#         return new_words
    
#     def random_swap(words, n):
#         if len(words) < 2:
#             return words
#         new_words = words.copy()
#         for _ in range(n):
#             try:
#                 idx1, idx2 = random.sample(range(len(new_words)), 2)
#                 new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
#             except Exception as e:
#                 print(f"Warning: random_swap failed with error {str(e)}")
#                 continue
#         return new_words
    
#     def random_deletion(words, p):
#         if len(words) <= 1:
#             return words
#         try:
#             new_words = [word for word in words if random.random() > p or not word.isalnum()]
#             if not new_words:
#                 return words
#             return new_words
#         except Exception as e:
#             print(f"Warning: random_deletion failed with error {str(e)}")
#             return words

#     # Split text into words and handle empty input
#     words = text.strip().split()
#     if not words:
#         return [text] * num_aug
    
#     num_words = len(words)
#     augmented_texts = []
    
#     # Calculate number of words to modify for each operation
#     n_sr = max(1, int(alpha * num_words))
#     n_ri = max(1, int(alpha * num_words))
#     n_rs = max(1, int(alpha * num_words))
    
#     operations = ['sr', 'ri', 'rs', 'rd']
#     fallback_suffixes = [
#         " por favor", " gracias", " si es posible", 
#         " urgente", " cuando pueda", " necesito ayuda",
#         " importante", " consulta", " duda", " solicitud"
#     ]
    
#     for _ in range(num_aug):
#         try:
#             operation = random.choice(operations)
            
#             if operation == 'sr':
#                 new_words = synonym_replacement(words, n_sr)
#             elif operation == 'ri':
#                 new_words = random_insertion(words, n_ri)
#             elif operation == 'rs':
#                 new_words = random_swap(words, n_rs)
#             else:  # rd
#                 new_words = random_deletion(words, alpha)
            
#             new_text = ' '.join(new_words)
#             if new_text.strip():  # Only add if not empty
#                 augmented_texts.append(new_text)
#             else:
#                 augmented_texts.append(text + random.choice(fallback_suffixes))
                
#         except Exception as e:
#             print(f"Warning: augmentation failed with error {str(e)}")
#             augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     # Ensure we have enough augmentations
#     while len(augmented_texts) < num_aug:
#         augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     return augmented_texts[:num_aug]  # Ensure we return exactly num_aug samples

# # def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5, seed=42):
# #     """Perform back-translation augmentation"""
# #     # Set seed
# #     random.seed(seed)
# #     np.random.seed(seed)
# #     torch.manual_seed(seed)

# #     augmented_texts = []
    
# #     try:
# #         # Translate to English
# #         inputs = tokenizer_to_en(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
# #         en_texts = model_to_en.generate(
# #             **inputs,
# #             num_beams=5,
# #             num_return_sequences=num_aug,
# #             max_length=512,
# #             temperature=0.8
# #         )
        
# #         # Translate back to Spanish/Portuguese
# #         for en_text in en_texts:
# #             decoded_en = tokenizer_to_en.decode(en_text, skip_special_tokens=True)
# #             inputs_es = tokenizer_to_es(decoded_en, return_tensors="pt", padding=True, truncation=True, max_length=512)
# #             es_text = model_to_es.generate(
# #                 **inputs_es,
# #                 num_beams=3,
# #                 max_length=512,
# #                 temperature=0.8
# #             )
# #             decoded_es = tokenizer_to_es.decode(es_text[0], skip_special_tokens=True)
# #             augmented_texts.append(decoded_es)
            
# #         return augmented_texts
# #     except Exception as e:
# #         print(f"Back-translation error: {str(e)}")
# #         # Fallback: return variations with added suffixes
# #         fallback_texts = []
# #         suffixes = [" por favor", " gracias", " si es posible", " urgente", " cuando pueda"]
# #         for suffix in suffixes[:num_aug]:
# #             fallback_texts.append(text + suffix)
# #         return fallback_texts

# def back_translation_augment(text, num_aug=5, seed=42):
#     """
#     Perform back-translation augmentation using Google Translate
#     Uses different intermediate languages to create variations
#     """
#     random.seed(seed)
    
#     # List of intermediate languages to create variations
#     intermediate_languages = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'gl']
#     source_lang = 'es'  # assuming Spanish is source language
    
#     augmented_texts = []
#     used_langs = random.sample(intermediate_languages, min(num_aug, len(intermediate_languages)))
    
#     for lang in used_langs:
#         try:
#             # Translate to intermediate language
#             translator_to = GoogleTranslator(source=source_lang, target=lang)
#             intermediate_text = translator_to.translate(text)
            
#             # Add small delay to avoid rate limiting
#             time.sleep(0.05)
            
#             # Translate back to Spanish
#             translator_back = GoogleTranslator(source=lang, target=source_lang)
#             back_translated = translator_back.translate(intermediate_text)
            
#             # Add small delay to avoid rate limiting
#             time.sleep(0.05)
            
#             if back_translated and back_translated.strip():
#                 augmented_texts.append(back_translated)
            
#         except Exception as e:
#             print(f"Translation error with language {lang}: {str(e)}")
#             continue
    
#     # If we couldn't generate enough augmentations, add variations of the original
#     fallback_suffixes = [
#         " por favor", " gracias", " si es posible", 
#         " urgente", " cuando pueda", " necesito ayuda",
#         " importante", " consulta", " duda", " solicitud"
#     ]
    
#     while len(augmented_texts) < num_aug:
#         augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     return augmented_texts[:num_aug]

# # def process_dataframe(df, topic_dict, method='EDA', config=None):
# #     """Process the dataframe and generate synthetic data"""
# #     # Setup
# #     save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
# #     topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
# #     if method == 'Back-translation':
# #         model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es = setup_language_models()
    
# #     # Process each row
# #     for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
# #         topic_name = row['topic_name']
# #         topic_num = get_key_by_value(topic_dict, topic_name)
        
# #         # Generate augmented texts
# #         if method == 'EDA':
# #             augmented_texts = eda_augment(row['text'])
# #         else:  # Back-translation
# #             augmented_texts = back_translation_augment(
# #                 row['text'], model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es
# #             )
        
# #         # Create and save new rows immediately
# #         for sample_num, aug_text in enumerate(augmented_texts, 1):
# #             new_row = {
# #                 'index_meta': row['index_meta'],
# #                 'topic_name': row['topic_name'],
# #                 'sample': sample_num,
# #                 'area_TEIS': row['area_TEIS'],
# #                 'text': aug_text
# #             }
# #             topic_dfs[topic_num].append_and_save(new_row)
    
# #     return topic_dfs

# def process_dataframe(df, topic_dict, method='EDA', config=None, seed=42):
#     """Process the dataframe and generate synthetic data"""
#     # Set seed
#     random.seed(seed)
#     np.random.seed(seed)
    
#     # Setup
#     save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
#     topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
#     # Process each row
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
#         # Set a different seed for each row
#         row_seed = seed + idx
#         random.seed(row_seed)
        
#         topic_name = row['topic_name']
#         topic_num = get_key_by_value(topic_dict, topic_name)

#         # Check if this index_meta already exists in the synthetic df
#         current_df = topic_dfs[topic_num].df
#         if row['index_meta'] in current_df['index_meta'].values:
#             print(f"Skipping {method} augmentation for index_meta {row['index_meta']} as it already exists")
#             continue
        
#         # Generate augmented texts
#         if method == 'EDA':
#             augmented_texts = eda_augment(row['text'], seed=row_seed)
#         else:  # Back-translation
#             augmented_texts = back_translation_augment(row['text'], seed=row_seed)
        
#         # Create and save new rows immediately
#         for sample_num, aug_text in enumerate(augmented_texts, 1):
#             new_row = {
#                 'index_meta': row['index_meta'],
#                 'topic_name': row['topic_name'],
#                 'sample': sample_num,
#                 'area_TEIS': row['area_TEIS'],
#                 'text': aug_text
#             }
#             topic_dfs[topic_num].append_and_save(new_row)
    
#     return topic_dfs

# def get_key_by_value(dictionary, search_value):
#     for key, value in dictionary.items():
#         if value == search_value:
#             return key
#     return None  # Return None if value not found

# def main(train_df, topic_dict, base_save_dir="synthetic_data", seed=42):
#     """Main function to run the entire pipeline"""
#     # Set initial seed
#     set_global_seeds(seed)

#     # Initialize configuration
#     config = Config(base_save_dir)
    
#     # Create folders
#     setup_folders(config)
    
#     # Process with EDA
#     print("Processing with EDA...")
#     eda_topic_dfs = process_dataframe(train_df, topic_dict, method='EDA', config=config, seed=seed)
    
#     # Process with Back-translation
#     print("Processing with Back-translation...")
#     bt_topic_dfs = process_dataframe(train_df, topic_dict, method='Back-translation', config=config, seed=seed)



In [38]:
# """For TREC-6"""
# # Add this at the beginning of the script
# class Config:
#     """Configuration class for save directories"""
#     def __init__(self, base_dir="synthetic_data"):
#         self.base_dir = base_dir
#         self.eda_dir = os.path.join(base_dir, "EDA")
#         self.bt_dir = os.path.join(base_dir, "Back-translation")

# def setup_folders(config):
#     """Create necessary folders for storing results"""
#     os.makedirs(config.base_dir, exist_ok=True)
#     os.makedirs(config.eda_dir, exist_ok=True)
#     os.makedirs(config.bt_dir, exist_ok=True)

# class TopicDataFrame:
#     """Class to handle topic dataframes with incremental saves in both CSV and PKL formats"""
#     def __init__(self, topic_num, save_dir):
#         self.topic_num = topic_num
#         self.save_dir = save_dir
#         self.csv_filename = os.path.join(save_dir, f"{topic_num}_synthetic.csv")
#         self.pkl_filename = os.path.join(save_dir, f"{topic_num}_synthetic.pkl")
#         self.columns = ['index_meta', 'TREC-6_Label', 'sample', 'text']
        
#         # Load existing data if any, otherwise create new DataFrame
#         if os.path.exists(self.pkl_filename):
#             with open(self.pkl_filename, 'rb') as f:
#                 self.df = pickle.load(f)
#         elif os.path.exists(self.csv_filename):
#             self.df = pd.read_csv(self.csv_filename)
#         else:
#             self.df = pd.DataFrame(columns=self.columns)
    
#     def append_and_save(self, new_row):
#         """Append a new row and save immediately in both formats"""
#         # Convert new_row dictionary to DataFrame
#         new_row_df = pd.DataFrame([new_row])
        
#         # Concatenate with existing DataFrame
#         self.df = pd.concat([self.df, new_row_df], ignore_index=True)
        
#         # Save as CSV
#         self.df.to_csv(self.csv_filename, index=False)
        
#         # Save as PKL
#         with open(self.pkl_filename, 'wb') as f:
#             pickle.dump(self.df, f)

# def initialize_topic_dfs(topic_dict, save_dir):
#     """Initialize TopicDataFrame objects for each topic"""
#     topic_dfs = {}
#     for topic_num in topic_dict.keys():
#         topic_dfs[topic_num] = TopicDataFrame(topic_num, save_dir)
#     return topic_dfs

# def setup_language_models():
#     """No need to setup models with deep-translator"""
#     return None

# # def eda_augment(text, num_aug=5):
# #     """Perform EDA augmentation for Spanish/Portuguese text"""
# #     # Initialize augmenter with Spanish WordNet
# #     aug = SynonymAug(aug_src='wordnet', lang='spa')  # Use 'por' for Portuguese
    
# #     try:
# #         augmented_texts = aug.augment(text, n=num_aug)
# #         return augmented_texts
# #     except:
# #         # Fallback: if augmentation fails, return slight modifications of original text
# #         return [text + " " + suffix for suffix in [
# #             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
# #         ]]

# # def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5):
# #     """Perform back-translation augmentation"""
# #     augmented_texts = []
    
# #     try:
# #         # Translate to English
# #         en_text = model_to_en.generate(
# #             **tokenizer_to_en(text, return_tensors="pt", padding=True),
# #             num_beams=5,
# #             num_return_sequences=num_aug,
# #             max_length=256
# #         )
        
# #         # Translate back to Spanish/Portuguese
# #         for en in en_text:
# #             es_text = model_to_es.generate(
# #                 **tokenizer_to_es(tokenizer_to_en.decode(en, skip_special_tokens=True), 
# #                                 return_tensors="pt", padding=True),
# #                 num_beams=1,
# #                 max_length=256
# #             )
# #             augmented_texts.append(tokenizer_to_es.decode(es_text[0], skip_special_tokens=True))
            
# #         return augmented_texts
# #     except:
# #         # Fallback: if translation fails, return slight modifications
# #         return [text + " " + suffix for suffix in [
# #             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
# #         ]]

# def eda_augment(text, num_aug=5, alpha=0.1, seed=42):
#     """
#     Perform Easy Data Augmentation for Spanish/Portuguese text with robust error handling
#     """
#     # Set seed
#     random.seed(seed)
#     np.random.seed(seed)

#     # Add input validation
#     if not isinstance(text, str) or not text.strip():
#         return [text] * num_aug
    
#     def get_synonyms(word, lang='spa'):  # Use 'por' for Portuguese
#         try:
#             aug = SynonymAug(aug_src='wordnet', lang=lang)
#             synonyms = aug.augment(word)[0]
#             return synonyms if isinstance(synonyms, list) else [synonyms]
#         except:
#             return [word]
    
#     def synonym_replacement(words, n):
#         if not words:
#             return words
#         new_words = words.copy()
#         random_word_list = list(set([word for word in words if word.isalnum()]))
#         if not random_word_list:  # If no alphanumeric words found
#             return words
#         n = min(n, len(random_word_list))
#         random_idx = random.sample(range(len(random_word_list)), n)
        
#         for idx in random_idx:
#             word = random_word_list[idx]
#             synonyms = get_synonyms(word)
#             if synonyms:
#                 random_syn = random.choice(synonyms)
#                 new_words = [random_syn if w == word else w for w in new_words]
#         return new_words
    
#     def random_insertion(words, n):
#         if not words:
#             return words
#         new_words = words.copy()
#         # Get list of alphanumeric words
#         alnum_words = [w for w in words if w.isalnum()]
#         if not alnum_words:  # If no alphanumeric words found
#             return words
            
#         for _ in range(n):
#             try:
#                 random_word = random.choice(alnum_words)
#                 synonyms = get_synonyms(random_word)
#                 if synonyms:
#                     random_syn = random.choice(synonyms)
#                     random_idx = random.randint(0, len(new_words))
#                     new_words.insert(random_idx, random_syn)
#             except Exception as e:
#                 print(f"Warning: random_insertion failed with error {str(e)}")
#                 continue
#         return new_words
    
#     def random_swap(words, n):
#         if len(words) < 2:
#             return words
#         new_words = words.copy()
#         for _ in range(n):
#             try:
#                 idx1, idx2 = random.sample(range(len(new_words)), 2)
#                 new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
#             except Exception as e:
#                 print(f"Warning: random_swap failed with error {str(e)}")
#                 continue
#         return new_words
    
#     def random_deletion(words, p):
#         if len(words) <= 1:
#             return words
#         try:
#             new_words = [word for word in words if random.random() > p or not word.isalnum()]
#             if not new_words:
#                 return words
#             return new_words
#         except Exception as e:
#             print(f"Warning: random_deletion failed with error {str(e)}")
#             return words

#     # Split text into words and handle empty input
#     words = text.strip().split()
#     if not words:
#         return [text] * num_aug
    
#     num_words = len(words)
#     augmented_texts = []
    
#     # Calculate number of words to modify for each operation
#     n_sr = max(1, int(alpha * num_words))
#     n_ri = max(1, int(alpha * num_words))
#     n_rs = max(1, int(alpha * num_words))
    
#     operations = ['sr', 'ri', 'rs', 'rd']
#     fallback_suffixes = [
#         " por favor", " gracias", " si es posible", 
#         " urgente", " cuando pueda", " necesito ayuda",
#         " importante", " consulta", " duda", " solicitud"
#     ]
    
#     for _ in range(num_aug):
#         try:
#             operation = random.choice(operations)
            
#             if operation == 'sr':
#                 new_words = synonym_replacement(words, n_sr)
#             elif operation == 'ri':
#                 new_words = random_insertion(words, n_ri)
#             elif operation == 'rs':
#                 new_words = random_swap(words, n_rs)
#             else:  # rd
#                 new_words = random_deletion(words, alpha)
            
#             new_text = ' '.join(new_words)
#             if new_text.strip():  # Only add if not empty
#                 augmented_texts.append(new_text)
#             else:
#                 augmented_texts.append(text + random.choice(fallback_suffixes))
                
#         except Exception as e:
#             print(f"Warning: augmentation failed with error {str(e)}")
#             augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     # Ensure we have enough augmentations
#     while len(augmented_texts) < num_aug:
#         augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     return augmented_texts[:num_aug]  # Ensure we return exactly num_aug samples

# # def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5, seed=42):
# #     """Perform back-translation augmentation"""
# #     # Set seed
# #     random.seed(seed)
# #     np.random.seed(seed)
# #     torch.manual_seed(seed)

# #     augmented_texts = []
    
# #     try:
# #         # Translate to English
# #         inputs = tokenizer_to_en(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
# #         en_texts = model_to_en.generate(
# #             **inputs,
# #             num_beams=5,
# #             num_return_sequences=num_aug,
# #             max_length=512,
# #             temperature=0.8
# #         )
        
# #         # Translate back to Spanish/Portuguese
# #         for en_text in en_texts:
# #             decoded_en = tokenizer_to_en.decode(en_text, skip_special_tokens=True)
# #             inputs_es = tokenizer_to_es(decoded_en, return_tensors="pt", padding=True, truncation=True, max_length=512)
# #             es_text = model_to_es.generate(
# #                 **inputs_es,
# #                 num_beams=3,
# #                 max_length=512,
# #                 temperature=0.8
# #             )
# #             decoded_es = tokenizer_to_es.decode(es_text[0], skip_special_tokens=True)
# #             augmented_texts.append(decoded_es)
            
# #         return augmented_texts
# #     except Exception as e:
# #         print(f"Back-translation error: {str(e)}")
# #         # Fallback: return variations with added suffixes
# #         fallback_texts = []
# #         suffixes = [" por favor", " gracias", " si es posible", " urgente", " cuando pueda"]
# #         for suffix in suffixes[:num_aug]:
# #             fallback_texts.append(text + suffix)
# #         return fallback_texts

# def back_translation_augment(text, num_aug=5, seed=42):
#     """
#     Perform back-translation augmentation using Google Translate
#     Uses different intermediate languages to create variations
#     """
#     random.seed(seed)
    
#     # List of intermediate languages to create variations
#     intermediate_languages = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'gl']
#     source_lang = 'es'  # assuming Spanish is source language
    
#     augmented_texts = []
#     used_langs = random.sample(intermediate_languages, min(num_aug, len(intermediate_languages)))
    
#     for lang in used_langs:
#         try:
#             # Translate to intermediate language
#             translator_to = GoogleTranslator(source=source_lang, target=lang)
#             intermediate_text = translator_to.translate(text)
            
#             # Add small delay to avoid rate limiting
#             time.sleep(0.05)
            
#             # Translate back to Spanish
#             translator_back = GoogleTranslator(source=lang, target=source_lang)
#             back_translated = translator_back.translate(intermediate_text)
            
#             # Add small delay to avoid rate limiting
#             time.sleep(0.05)
            
#             if back_translated and back_translated.strip():
#                 augmented_texts.append(back_translated)
            
#         except Exception as e:
#             print(f"Translation error with language {lang}: {str(e)}")
#             continue
    
#     # If we couldn't generate enough augmentations, add variations of the original
#     fallback_suffixes = [
#         " por favor", " gracias", " si es posible", 
#         " urgente", " cuando pueda", " necesito ayuda",
#         " importante", " consulta", " duda", " solicitud"
#     ]
    
#     while len(augmented_texts) < num_aug:
#         augmented_texts.append(text + random.choice(fallback_suffixes))
    
#     return augmented_texts[:num_aug]

# # def process_dataframe(df, topic_dict, method='EDA', config=None):
# #     """Process the dataframe and generate synthetic data"""
# #     # Setup
# #     save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
# #     topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
# #     if method == 'Back-translation':
# #         model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es = setup_language_models()
    
# #     # Process each row
# #     for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
# #         topic_name = row['topic_name']
# #         topic_num = get_key_by_value(topic_dict, topic_name)
        
# #         # Generate augmented texts
# #         if method == 'EDA':
# #             augmented_texts = eda_augment(row['text'])
# #         else:  # Back-translation
# #             augmented_texts = back_translation_augment(
# #                 row['text'], model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es
# #             )
        
# #         # Create and save new rows immediately
# #         for sample_num, aug_text in enumerate(augmented_texts, 1):
# #             new_row = {
# #                 'index_meta': row['index_meta'],
# #                 'topic_name': row['topic_name'],
# #                 'sample': sample_num,
# #                 'area_TEIS': row['area_TEIS'],
# #                 'text': aug_text
# #             }
# #             topic_dfs[topic_num].append_and_save(new_row)
    
# #     return topic_dfs

# def process_dataframe(df, topic_dict, method='EDA', config=None, seed=42):
#     """Process the dataframe and generate synthetic data"""
#     # Set seed
#     random.seed(seed)
#     np.random.seed(seed)
    
#     # Setup
#     save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
#     topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
#     # Process each row
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
#         # Set a different seed for each row
#         row_seed = seed + idx
#         random.seed(row_seed)
        
#         topic_name = row['TREC-6_Label']
#         topic_num = get_key_by_value(topic_dict, topic_name)

#         # Check if this index_meta already exists in the synthetic df
#         current_df = topic_dfs[topic_num].df
#         if row['index_meta'] in current_df['index_meta'].values:
#             print(f"Skipping {method} augmentation for index_meta {row['index_meta']} as it already exists")
#             continue
        
#         # Generate augmented texts
#         if method == 'EDA':
#             augmented_texts = eda_augment(row['text'], seed=row_seed)
#         else:  # Back-translation
#             augmented_texts = back_translation_augment(row['text'], seed=row_seed)
        
#         # Create and save new rows immediately
#         for sample_num, aug_text in enumerate(augmented_texts, 1):
#             new_row = {
#                 'index_meta': row['index_meta'],
#                 'TREC-6_Label': row['TREC-6_Label'],
#                 'sample': sample_num,
#                 # 'area_TEIS': row['area_TEIS'],
#                 'text': aug_text
#             }
#             topic_dfs[topic_num].append_and_save(new_row)
    
#     return topic_dfs

# def get_key_by_value(dictionary, search_value):
#     for key, value in dictionary.items():
#         if value == search_value:
#             return key
#     return None  # Return None if value not found

# def main(train_df, topic_dict, base_save_dir="synthetic_data", seed=42):
#     """Main function to run the entire pipeline"""
#     # Set initial seed
#     set_global_seeds(seed)

#     # Initialize configuration
#     config = Config(base_save_dir)
    
#     # Create folders
#     setup_folders(config)
    
#     # Process with EDA
#     print("Processing with EDA...")
#     eda_topic_dfs = process_dataframe(train_df, topic_dict, method='EDA', config=config, seed=seed)
    
#     # Process with Back-translation
#     print("Processing with Back-translation...")
#     bt_topic_dfs = process_dataframe(train_df, topic_dict, method='Back-translation', config=config, seed=seed)



In [39]:
"""For Amazon Review"""
# Add this at the beginning of the script
class Config:
    """Configuration class for save directories"""
    def __init__(self, base_dir="synthetic_data"):
        self.base_dir = base_dir
        self.eda_dir = os.path.join(base_dir, "EDA")
        self.bt_dir = os.path.join(base_dir, "Back-translation")

def setup_folders(config):
    """Create necessary folders for storing results"""
    os.makedirs(config.base_dir, exist_ok=True)
    os.makedirs(config.eda_dir, exist_ok=True)
    os.makedirs(config.bt_dir, exist_ok=True)

class TopicDataFrame:
    """Class to handle topic dataframes with incremental saves in both CSV and PKL formats"""
    def __init__(self, topic_num, save_dir):
        self.topic_num = topic_num
        self.save_dir = save_dir
        self.csv_filename = os.path.join(save_dir, f"{topic_num}_synthetic.csv")
        self.pkl_filename = os.path.join(save_dir, f"{topic_num}_synthetic.pkl")
        self.columns = ['index_meta', 'rating', 'sample', 'title', 'text']
        
        # Load existing data if any, otherwise create new DataFrame
        if os.path.exists(self.pkl_filename):
            with open(self.pkl_filename, 'rb') as f:
                self.df = pickle.load(f)
        elif os.path.exists(self.csv_filename):
            self.df = pd.read_csv(self.csv_filename)
        else:
            self.df = pd.DataFrame(columns=self.columns)
    
    def append_and_save(self, new_row):
        """Append a new row and save immediately in both formats"""
        # Convert new_row dictionary to DataFrame
        new_row_df = pd.DataFrame([new_row])
        
        # Concatenate with existing DataFrame
        self.df = pd.concat([self.df, new_row_df], ignore_index=True)
        
        # Save as CSV
        self.df.to_csv(self.csv_filename, index=False)
        
        # Save as PKL
        with open(self.pkl_filename, 'wb') as f:
            pickle.dump(self.df, f)

def initialize_topic_dfs(topic_dict, save_dir):
    """Initialize TopicDataFrame objects for each topic"""
    topic_dfs = {}
    for topic_num in topic_dict.keys():
        topic_dfs[topic_num] = TopicDataFrame(topic_num, save_dir)
    return topic_dfs

def setup_language_models():
    """No need to setup models with deep-translator"""
    return None

# def eda_augment(text, num_aug=5):
#     """Perform EDA augmentation for Spanish/Portuguese text"""
#     # Initialize augmenter with Spanish WordNet
#     aug = SynonymAug(aug_src='wordnet', lang='spa')  # Use 'por' for Portuguese
    
#     try:
#         augmented_texts = aug.augment(text, n=num_aug)
#         return augmented_texts
#     except:
#         # Fallback: if augmentation fails, return slight modifications of original text
#         return [text + " " + suffix for suffix in [
#             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
#         ]]

# def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5):
#     """Perform back-translation augmentation"""
#     augmented_texts = []
    
#     try:
#         # Translate to English
#         en_text = model_to_en.generate(
#             **tokenizer_to_en(text, return_tensors="pt", padding=True),
#             num_beams=5,
#             num_return_sequences=num_aug,
#             max_length=256
#         )
        
#         # Translate back to Spanish/Portuguese
#         for en in en_text:
#             es_text = model_to_es.generate(
#                 **tokenizer_to_es(tokenizer_to_en.decode(en, skip_special_tokens=True), 
#                                 return_tensors="pt", padding=True),
#                 num_beams=1,
#                 max_length=256
#             )
#             augmented_texts.append(tokenizer_to_es.decode(es_text[0], skip_special_tokens=True))
            
#         return augmented_texts
#     except:
#         # Fallback: if translation fails, return slight modifications
#         return [text + " " + suffix for suffix in [
#             "por favor", "gracias", "si es posible", "urgente", "cuando pueda"
#         ]]

def eda_augment(text, num_aug=5, alpha=0.1, seed=42):
    """
    Perform Easy Data Augmentation for Spanish/Portuguese text with robust error handling
    """
    # Set seed
    random.seed(seed)
    np.random.seed(seed)

    # Add input validation
    if not isinstance(text, str) or not text.strip():
        return [text] * num_aug
    
    def get_synonyms(word, lang='spa'):  # Use 'por' for Portuguese
        try:
            aug = SynonymAug(aug_src='wordnet', lang=lang)
            synonyms = aug.augment(word)[0]
            return synonyms if isinstance(synonyms, list) else [synonyms]
        except:
            return [word]
    
    def synonym_replacement(words, n):
        if not words:
            return words
        new_words = words.copy()
        random_word_list = list(set([word for word in words if word.isalnum()]))
        if not random_word_list:  # If no alphanumeric words found
            return words
        n = min(n, len(random_word_list))
        random_idx = random.sample(range(len(random_word_list)), n)
        
        for idx in random_idx:
            word = random_word_list[idx]
            synonyms = get_synonyms(word)
            if synonyms:
                random_syn = random.choice(synonyms)
                new_words = [random_syn if w == word else w for w in new_words]
        return new_words
    
    def random_insertion(words, n):
        if not words:
            return words
        new_words = words.copy()
        # Get list of alphanumeric words
        alnum_words = [w for w in words if w.isalnum()]
        if not alnum_words:  # If no alphanumeric words found
            return words
            
        for _ in range(n):
            try:
                random_word = random.choice(alnum_words)
                synonyms = get_synonyms(random_word)
                if synonyms:
                    random_syn = random.choice(synonyms)
                    random_idx = random.randint(0, len(new_words))
                    new_words.insert(random_idx, random_syn)
            except Exception as e:
                print(f"Warning: random_insertion failed with error {str(e)}")
                continue
        return new_words
    
    def random_swap(words, n):
        if len(words) < 2:
            return words
        new_words = words.copy()
        for _ in range(n):
            try:
                idx1, idx2 = random.sample(range(len(new_words)), 2)
                new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
            except Exception as e:
                print(f"Warning: random_swap failed with error {str(e)}")
                continue
        return new_words
    
    def random_deletion(words, p):
        if len(words) <= 1:
            return words
        try:
            new_words = [word for word in words if random.random() > p or not word.isalnum()]
            if not new_words:
                return words
            return new_words
        except Exception as e:
            print(f"Warning: random_deletion failed with error {str(e)}")
            return words

    # Split text into words and handle empty input
    words = text.strip().split()
    if not words:
        return [text] * num_aug
    
    num_words = len(words)
    augmented_texts = []
    
    # Calculate number of words to modify for each operation
    n_sr = max(1, int(alpha * num_words))
    n_ri = max(1, int(alpha * num_words))
    n_rs = max(1, int(alpha * num_words))
    
    operations = ['sr', 'ri', 'rs', 'rd']
    fallback_suffixes = [
        " por favor", " gracias", " si es posible", 
        " urgente", " cuando pueda", " necesito ayuda",
        " importante", " consulta", " duda", " solicitud"
    ]
    
    for _ in range(num_aug):
        try:
            operation = random.choice(operations)
            
            if operation == 'sr':
                new_words = synonym_replacement(words, n_sr)
            elif operation == 'ri':
                new_words = random_insertion(words, n_ri)
            elif operation == 'rs':
                new_words = random_swap(words, n_rs)
            else:  # rd
                new_words = random_deletion(words, alpha)
            
            new_text = ' '.join(new_words)
            if new_text.strip():  # Only add if not empty
                augmented_texts.append(new_text)
            else:
                augmented_texts.append(text + random.choice(fallback_suffixes))
                
        except Exception as e:
            print(f"Warning: augmentation failed with error {str(e)}")
            augmented_texts.append(text + random.choice(fallback_suffixes))
    
    # Ensure we have enough augmentations
    while len(augmented_texts) < num_aug:
        augmented_texts.append(text + random.choice(fallback_suffixes))
    
    return augmented_texts[:num_aug]  # Ensure we return exactly num_aug samples

# def back_translation_augment(text, model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es, num_aug=5, seed=42):
#     """Perform back-translation augmentation"""
#     # Set seed
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)

#     augmented_texts = []
    
#     try:
#         # Translate to English
#         inputs = tokenizer_to_en(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         en_texts = model_to_en.generate(
#             **inputs,
#             num_beams=5,
#             num_return_sequences=num_aug,
#             max_length=512,
#             temperature=0.8
#         )
        
#         # Translate back to Spanish/Portuguese
#         for en_text in en_texts:
#             decoded_en = tokenizer_to_en.decode(en_text, skip_special_tokens=True)
#             inputs_es = tokenizer_to_es(decoded_en, return_tensors="pt", padding=True, truncation=True, max_length=512)
#             es_text = model_to_es.generate(
#                 **inputs_es,
#                 num_beams=3,
#                 max_length=512,
#                 temperature=0.8
#             )
#             decoded_es = tokenizer_to_es.decode(es_text[0], skip_special_tokens=True)
#             augmented_texts.append(decoded_es)
            
#         return augmented_texts
#     except Exception as e:
#         print(f"Back-translation error: {str(e)}")
#         # Fallback: return variations with added suffixes
#         fallback_texts = []
#         suffixes = [" por favor", " gracias", " si es posible", " urgente", " cuando pueda"]
#         for suffix in suffixes[:num_aug]:
#             fallback_texts.append(text + suffix)
#         return fallback_texts

def back_translation_augment(text, num_aug=5, seed=42):
    """
    Perform back-translation augmentation using Google Translate
    Uses different intermediate languages to create variations
    """
    random.seed(seed)
    
    # List of intermediate languages to create variations
    intermediate_languages = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'gl']
    source_lang = 'es'  # assuming Spanish is source language
    
    augmented_texts = []
    used_langs = random.sample(intermediate_languages, min(num_aug, len(intermediate_languages)))
    
    for lang in used_langs:
        try:
            # Translate to intermediate language
            translator_to = GoogleTranslator(source=source_lang, target=lang)
            intermediate_text = translator_to.translate(text)
            
            # Add small delay to avoid rate limiting
            time.sleep(0.05)
            
            # Translate back to Spanish
            translator_back = GoogleTranslator(source=lang, target=source_lang)
            back_translated = translator_back.translate(intermediate_text)
            
            # Add small delay to avoid rate limiting
            time.sleep(0.05)
            
            if back_translated and back_translated.strip():
                augmented_texts.append(back_translated)
            
        except Exception as e:
            print(f"Translation error with language {lang}: {str(e)}")
            continue
    
    # If we couldn't generate enough augmentations, add variations of the original
    fallback_suffixes = [
        " por favor", " gracias", " si es posible", 
        " urgente", " cuando pueda", " necesito ayuda",
        " importante", " consulta", " duda", " solicitud"
    ]
    
    while len(augmented_texts) < num_aug:
        augmented_texts.append(text + random.choice(fallback_suffixes))
    
    return augmented_texts[:num_aug]

# def process_dataframe(df, topic_dict, method='EDA', config=None):
#     """Process the dataframe and generate synthetic data"""
#     # Setup
#     save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
#     topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
#     if method == 'Back-translation':
#         model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es = setup_language_models()
    
#     # Process each row
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
#         topic_name = row['topic_name']
#         topic_num = get_key_by_value(topic_dict, topic_name)
        
#         # Generate augmented texts
#         if method == 'EDA':
#             augmented_texts = eda_augment(row['text'])
#         else:  # Back-translation
#             augmented_texts = back_translation_augment(
#                 row['text'], model_to_en, model_to_es, tokenizer_to_en, tokenizer_to_es
#             )
        
#         # Create and save new rows immediately
#         for sample_num, aug_text in enumerate(augmented_texts, 1):
#             new_row = {
#                 'index_meta': row['index_meta'],
#                 'topic_name': row['topic_name'],
#                 'sample': sample_num,
#                 'area_TEIS': row['area_TEIS'],
#                 'text': aug_text
#             }
#             topic_dfs[topic_num].append_and_save(new_row)
    
#     return topic_dfs

def process_dataframe(df, topic_dict, method='EDA', config=None, seed=42):
    """Process the dataframe and generate synthetic data"""
    # Set seed
    random.seed(seed)
    np.random.seed(seed)
    
    # Setup
    save_dir = config.eda_dir if method == 'EDA' else config.bt_dir
    topic_dfs = initialize_topic_dfs(topic_dict, save_dir)
    
    # Process each row
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {method}"):
        # Set a different seed for each row
        row_seed = seed + idx
        random.seed(row_seed)
        
        topic_name = row['rating']
        topic_num = get_key_by_value(topic_dict, topic_name)

        # Check if this index_meta already exists in the synthetic df
        current_df = topic_dfs[topic_num].df
        if row['index_meta'] in current_df['index_meta'].values:
            print(f"Skipping {method} augmentation for index_meta {row['index_meta']} as it already exists")
            continue
        
        # Generate augmented texts
        if method == 'EDA':
            augmented_texts = eda_augment(row['text'], seed=row_seed)
        else:  # Back-translation
            augmented_texts = back_translation_augment(row['text'], seed=row_seed)
        
        # Create and save new rows immediately
        for sample_num, aug_text in enumerate(augmented_texts, 1):
            new_row = {
                'index_meta': row['index_meta'],
                'rating': row['rating'],
                'sample': sample_num,
                'title': row['title'],
                'text': aug_text
            }
            topic_dfs[topic_num].append_and_save(new_row)
    
    return topic_dfs

def get_key_by_value(dictionary, search_value):
    for key, value in dictionary.items():
        if value == search_value:
            return key
    return None  # Return None if value not found

def main(train_df, topic_dict, base_save_dir="synthetic_data", seed=42):
    """Main function to run the entire pipeline"""
    # Set initial seed
    set_global_seeds(seed)

    # Initialize configuration
    config = Config(base_save_dir)
    
    # Create folders
    setup_folders(config)
    
    # Process with EDA
    print("Processing with EDA...")
    eda_topic_dfs = process_dataframe(train_df, topic_dict, method='EDA', config=config, seed=seed)
    
    # Process with Back-translation
    print("Processing with Back-translation...")
    bt_topic_dfs = process_dataframe(train_df, topic_dict, method='Back-translation', config=config, seed=seed)



In [40]:
# Usage example:
if __name__ == "__main__":
    # Specify your save directory here
    BASE_SAVE_DIR = "D:/Oxford/Second Year/AutoGeTS_Revisions/Additional_Datasets/Amazon_Review/Synthetic_Data"
    SEED = 42
    
    # Run the pipeline
    main(train_PCA_YZ_df, topic_dict, base_save_dir=BASE_SAVE_DIR, seed=SEED)

Processing with EDA...


Processing EDA:   0%|          | 0/6400 [00:00<?, ?it/s]

Processing EDA: 100%|██████████| 6400/6400 [46:43<00:00,  2.28it/s]  


Processing with Back-translation...


Processing Back-translation:   1%|          | 56/6400 [05:22<10:09:31,  5.76s/it]


KeyboardInterrupt: 